In [5]:
import os
from authenticate_service_account import *
from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd


In [6]:
key_path = 'raw_data/skyscanner-insights-343713-06a72b2724ca.json'

# [START bigquery_client_json_credentials]

# TODO(developer): Set key_path to the path to the service account key
#                  file.
# key_path = "path/to/service_account.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
# [END bigquery_client_json_credentials]


In [7]:
datasets = client.list_datasets()

for dataset in datasets:
  print(dataset)


In [8]:
# query = '''SELECT * FROM `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
# WHERE Timestamp BETWEEN DATETIME("2023-08-17")
# AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY) and Leg_0_StopsCount is not null'''

query = '''Select * from `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
WHERE Timestamp BETWEEN DATETIME("2023-08-17")AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY) and Leg_0_StopsCount is not null
ORDER BY RAND()
LIMIT 200
'''


In [9]:
query_job = client.query(query)


In [10]:
results = query_job.result()


In [11]:
the_data = results.to_dataframe()


In [12]:
the_data.shape


(200, 153)

In [11]:
# data_q1 = the_data[:824792]
# data_q2 = the_data[824792:1649584]
# data_q3 = the_data[1649584:2474376]
# data_q4 = the_data[2474376:]

# data_q4.columns

# data_q1.to_json("raw_data/data_25", orient='records')
# test_df = pd.read_json('raw_data/data_25', orient='records')
